Hi everyone! This is the notebook we will use tf.Keras to train/evaluate/predict a classifer for MNIST with convolutional layers. With tf.Keras things become very simple all we have to do is to combine codes from previous notebooks!

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import utils
print(tf.__version__)

In [ ]:
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

Preparing Data

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype('float32').reshape((-1,28,28,1))
x_test = x_test.astype('float32').reshape((-1,28,28,1))
x_train /= 255.0
x_test /= 255.0
trainDataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
testDataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [ ]:
utils.show_images(trainDataset,class_names)

Preparing Model

In [ ]:
cnnModel = keras.Sequential()
cnnModel.add(layers.Conv2D(input_shape=[28,28,1],filters = 32, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 32, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 2, strides = 2))
cnnModel.add(layers.Dropout(0.25))
cnnModel.add(layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 2, strides = 2))
cnnModel.add(layers.Dropout(0.25))
cnnModel.add(layers.Flatten())
cnnModel.add(layers.Dense(units = 512, activation = 'relu'))
cnnModel.add(layers.Dense(units = 10, activation = 'softmax'))
cnnModel.summary()

Compiling Model

In [ ]:
cnnModel.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Training Procedure

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')
]
history = cnnModel.fit(trainDataset.shuffle(1000).batch(64), epochs=20, validation_data=testDataset.batch(64), validation_steps=100, callbacks = callbacks)

In [ ]:
# history record the accuracy/loss on training set/validation set after every epoch 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'valivation'], loc='upper left')
plt.show()

You must set the batch_size for the dataset before sending it to the model for training/evaluation.

In [ ]:
result = cnnModel.evaluate(testDataset.batch(64))

In [ ]:
result

Prediction Phase

In [ ]:
predict_result = tf.argmax(cnnModel.predict(x_test[:9]),axis=1)

In [ ]:
predict_result.numpy()

In [ ]:
utils.show_images(testDataset, class_names, predict_result)